In [ ]:
import os
from pathlib import Path

current_dir = Path('./')
data_dir = current_dir / 'speech_commands_v0.01'
train_file_path = data_dir / "train_digit_list.txt"
test_file_path = data_dir / "testing_digit_list.txt"

train_file = open(train_file_path, "r")
training_list = [data_dir / x for x in train_file.read().splitlines()]

test_file = open(test_file_path, "r")
testing_list = [data_dir / x for x in test_file.read().splitlines()]

In [ ]:
import os
from pathlib import Path
import numpy as np

with open('X_train_original.npy', 'rb') as f:
    X_train_org = np.load(f)

with open('X_train_reverb_random.npy', 'rb') as f:
    X_train_reverb = np.load(f)
    
with open('X_test_reverb_random.npy', 'rb') as f:
    X_test_reverb = np.load(f)

In [ ]:
import scipy.io.wavfile as wav

sample_num = 25
wav.write('./audio/tas_original.wav',  16000, X_train_org[sample_num].astype(np.int16))
wav.write('./audio/tas_reverb.wav', 16000, X_train_reverb[sample_num].astype(np.int16))

In [ ]:
!asteroid-infer "mhu-coder/ConvTasNet_Libri1Mix_enhsingle" --files "./Audio/tas_reverb.wav"

In [ ]:
import librosa

def plot_stft(prefix):
    original_file = './audio/{}_original.wav'.format(prefix)
    reverb_file = './audio/{}_reverb.wav'.format(prefix)
    denoise_file = './audio/{}_reverb_est1.wav'.format(prefix)
    
    o = librosa.load(original_file, 16000)[0]
    r = librosa.load(reverb_file, 16000)[0]
    d = librosa.load(denoise_file, 16000)[0]
    
    print(np.min(r), np.max(r))
        
    O = librosa.amplitude_to_db(librosa.stft(o), ref=np.max)
    R = librosa.amplitude_to_db(librosa.stft(r), ref=np.max)
    D = librosa.amplitude_to_db(librosa.stft(d), ref=np.max)
    
    return O, R, D

In [ ]:
o, r, d = plot_stft('tas')

In [ ]:
import librosa.display
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
librosa.display.specshow(d, y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('ConvNetTas')
plt.savefig('img/tas_fix.png')
plt.show()

# Do filter on all signal

In [ ]:
from tqdm.notebook import tnrange
import os

train_data = []
for i in tnrange(len(X_train_reverb)):
    wav.write('./audio/train.wav',  16000, X_train_reverb[i].astype(np.int16))
    !eval {"asteroid-infer 'mhu-coder/ConvTasNet_Libri1Mix_enhsingle' --files './Audio/train.wav'"}
    data = librosa.load('./Audio/train_est1.wav', 16000)[0]
    os.remove('./Audio/train_est1.wav')
    train_data.append(data)
    
train_data = np.asarray(train_data)

In [ ]:
save_dir = Path('./E2E_data')
np.save(save_dir / "ConvTasnet_train_reverb.npy", train_data)

In [ ]:
from tqdm.notebook import tnrange
import os

test_data = []
for i in tnrange(len(X_train_reverb)):
    wav.write('./audio/test.wav',  16000, X_test_reverb[i].astype(np.int16))
    !eval {"asteroid-infer 'mhu-coder/ConvTasNet_Libri1Mix_enhsingle' --files './Audio/test.wav'"}
    data = librosa.load('./Audio/test_est1.wav', 16000)[0]
    os.remove('./Audio/test_est1.wav')
    test_data.append(data)
    
test_data = np.asarray(test_data)

In [ ]:
save_dir = Path('./E2E_data')
np.save(save_dir / "ConvTasnet_test_reverb.npy", test_data)